#### **Extracting features of 2018 movies from Wikipedia**

In [4]:
import pandas as pd
import numpy as np

In [5]:
import requests
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response = requests.get(link, headers=headers)
df1 = pd.read_html(response.text, header=0)[2]
df2 = pd.read_html(response.text, header=0)[3]
df3 = pd.read_html(response.text, header=0)[4]
df4 = pd.read_html(response.text, header=0)[5]

C:\Users\HARSH PRATAP SINGH\AppData\Local\Temp\ipykernel_26968\2284562741.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df1 = pd.read_html(response.text, header=0)[2]
C:\Users\HARSH PRATAP SINGH\AppData\Local\Temp\ipykernel_26968\2284562741.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df2 = pd.read_html(response.text, header=0)[3]
C:\Users\HARSH PRATAP SINGH\AppData\Local\Temp\ipykernel_26968\2284562741.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df3 = pd.read_html(response.text, header=0)[4]
C:\Users\HARSH PRATAP SINGH\AppData\Local\Temp\ipykernel_26968\2284562741.py:10: FutureWarning: Passing li

In [6]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True)
df


,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2]
1,J A N U A R Y,5,The Strange Ones,Vertical Entertainment,Christopher Radcliff (director/screenplay); La...,[3]
2,J A N U A R Y,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[4]
3,J A N U A R Y,12,Proud Mary,Screen Gems,"Babak Najafi (director); John S. Newman, Chris...",[5]
4,J A N U A R Y,12,Acts of Violence,Lionsgate Premiere,Brett Donowho (director); Nicolas Aaron Mezzan...,[6]
...,...,...,...,...,...,...
245,D E C E M B E R,21,Second Act,STX Entertainment,"Peter Segal (director); Justin Zackham, Elaine...",[238]
246,D E C E M B E R,25,Holmes & Watson,Columbia Pictures / Gary Sanchez Productions /...,Etan Cohen (director/screenplay); Will Ferrell...,[142]
247,D E C E M B E R,25,Vice,Annapurna Pictures / Plan B Entertainment,Adam McKay (director/screenplay); Christian Ba...,[117]
248,D E C E M B E R,25,On the Basis of Sex,Focus Features,Mimi Leder (director); Daniel Stiepleman (scre...,[207]


As it is observed in the above table, we do not have the **genre** column so we will get the genre data from IMDb. You will need to create an API key in order to access the API key.

In [14]:
pip install tmdbv3api

Note: you may need to restart the kernel to use updated packages.


In [1]:
#get your API Key here: https://www.themoviedb.org/settings/api
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '84e4625cb7b1b33fd2e09321faac01d2' 

In [ ]:
import time
from tmdbv3api import Search
import requests

search = Search()
genre_cache = {}

def get_genre(title):
    # empty / nan guard
    if not title or title == 'nan':
        return None

    # cache to avoid repeat API calls
    if title in genre_cache:
        return genre_cache[title]

    try:
        result = search.movies(title)

        if not result:
            genre_cache[title] = None
            return None

        movie_id = result[0].id
        response = requests.get(
            f"https://api.themoviedb.org/3/movie/{movie_id}",
            params={"api_key": tmdb.api_key},
            timeout=10
        )

        genres = [g['name'] for g in response.json().get('genres', [])]
        genre_cache[title] = genres

        time.sleep(0.25)  # ⭐ CRITICAL: rate limiting
        return genres

    except Exception as e:
        print(f"Error for {title}: {e}")
        return None


In [ ]:
df['genres'] = df['Title'].apply(get_genre)
df

In [ ]:
df_2018 = df[['Title','Cast and crew','genres']]
df_2018

**Extract Directors**

In [ ]:
def get_director(x):
    if " (director)" in x: #we are getting "director"
        return x.split(" (director)")[0]
    elif " (directors)" in x: #we are getting "directors" with "s"
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0] #we are getting "directors/screenplay"

In [ ]:

#let's apply the above function on the "Cast and crew" column
df_2018['director_name'] = df_2018['Cast and crew'].map(lambda x: get_director(x))


In [ ]:
df_2018

**Actor 1**

In [ ]:

def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [ ]:

df_2018['actor_1_name'] = df_2018['Cast and crew'].map(lambda x: get_actor1(x))


**Actor 2**

In [ ]:

def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [ ]:

df_2018['actor_2_name'] = df_2018['Cast and crew'].map(lambda x: get_actor2(x))


**Actor 3**

In [ ]:

def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [ ]:

df_2018['actor_3_name'] = df_2018['Cast and crew'].map(lambda x: get_actor3(x))


In [ ]:

df_2018

In [ ]:

df_2018 = df_2018.rename(columns={'Title':'movie_title'})


In [ ]:

new_df18 = df_2018.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]


In [ ]:

new_df18

In [ ]:
new_df18['actor_2_name'] = new_df18['actor_2_name'].replace(np.nan, 'unknown')
new_df18['actor_3_name'] = new_df18['actor_3_name'].replace(np.nan, 'unknown')

In [ ]:

new_df18['movie_title'] = new_df18['movie_title'].str.lower()


In [ ]:

new_df18['comb'] = new_df18['actor_1_name'] + ' ' + new_df18['actor_2_name'] + ' '+ new_df18['actor_3_name'] + ' '+ new_df18['director_name'] +' ' + new_df18['genres']
new_df18

###**Extracting features of 2019 movies from Wikipedia**

In [ ]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [ ]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [ ]:
df

In [ ]:
df['genres'] = df['Title'].map(lambda x: get_genre(str(x)))


In [ ]:
df_2019 = df[['Title','Cast and crew','genres']]
df_2019

In [ ]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [ ]:
df_2019['director_name'] = df_2019['Cast and crew'].map(lambda x: get_director(str(x)))


In [ ]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [ ]:
df_2019['actor_1_name'] = df_2019['Cast and crew'].map(lambda x: get_actor1(x))


In [ ]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [ ]:
df_2019['actor_2_name'] = df_2019['Cast and crew'].map(lambda x: get_actor2(x))


In [ ]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [ ]:
df_2019['actor_3_name'] = df_2019['Cast and crew'].map(lambda x: get_actor3(x))


In [ ]:
df_2019 = df_2019.rename(columns={'Title':'movie_title'})


In [ ]:
new_df19 = df_2019.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]


In [ ]:
new_df19['actor_2_name'] = new_df19['actor_2_name'].replace(np.nan, 'unknown')
new_df19['actor_3_name'] = new_df19['actor_3_name'].replace(np.nan, 'unknown')

In [ ]:
new_df19['movie_title'] = new_df19['movie_title'].str.lower()


In [ ]:
new_df19['comb'] = new_df19['actor_1_name'] + ' ' + new_df19['actor_2_name'] + ' '+ new_df19['actor_3_name'] + ' '+ new_df19['director_name'] +' ' + new_df19['genres']


In [ ]:
new_df19

In [ ]:
my_df = new_df18.append(new_df19,ignore_index=True)
my_df

In [ ]:
old_df = pd.read_csv('/content/new_data.csv')
old_df

In [ ]:
final_df = old_df.append(my_df,ignore_index=True)
final_df

In [ ]:
final_df.isna().sum()

In [ ]:
final_df = final_df.dropna(how='any')

In [ ]:
final_df.isna().sum()

In [ ]:
final_df.to_csv('final_data.csv',index=False)